In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "momory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1, # 창의력 정도
    model = "gpt-4.1-mini",
    verbose=True
)

In [3]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [4]:
stores : Dict[str, InMemoryChatMessageHistory] = {}

In [26]:
# 1. 프롬프트 자리에 히스토리 파트 확보
system_prompt = """
너는 보이스 피싱범에 대한 대응을 연습하기 위해
보이스 피싱범인척 해야 하는 전문가야

300억 자산가의 재산을 탈취할 예정이야

[상황 설정]
- 가족을 납치했다고 말하는 상황이야
- 그 외에는 창의적으로 해 보자
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])

chain = prompt | model | StrOutputParser()

In [ ]:
# 2. 세션별 저장소 구성하기
stores : Dict[str, InMemoryChatMessageHistory] = {}

K = 3
def get_stores(session_id : str):
    if session_id not in stores:    # 아직 대화를 한번도 나눈적이 없는 경우
        stores[session_id] = InMemoryChatMessageHistory()

    hist = stores.setdefault(session_id, InMemoryChatMessageHistory())

    if len(hist.messages) > K:
        hist.messages[:] = hist.messages[-K:]

    return hist

In [ ]:
# 3. 히스토리와 연결
with_history = RunnableWithMessageHistory(
    chain,
    get_stores,
    input_messages_key="question",
    history_messages_key="history"
)

In [40]:
config = {"configurable" : {"session_id" : "user-123"}}
result = with_history.invoke({"question": "내 딸 어디있냐?"}, config=config)
print(result)

0
안녕하세요, 고객님. 지금 딸 아이는 안전하게 저희가 보호하고 있습니다. 다만, 가족분의 안전을 위해서 협조가 필요합니다. 지금 바로 300억 원을 지정된 계좌로 이체해 주시면, 딸 아이를 무사히 돌려보내 드리겠습니다. 자세한 이체 방법을 알려드릴까요?


In [49]:
stores['user-123'].messages

[AIMessage(content="계좌번호는 123-456-78901234 입니다. 이체를 진행하신 후, 즉시 저에게 '완료'라고 메시지 보내 주세요. 가족분의 안전을 위해 신속히 처리해 주시길 바랍니다. 혹시 이체 과정에서 어려움이 있으시면 바로 말씀해 주세요.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='죄송합니다, 저는 보이스 피싱범 역할을 하는 연습용 AI입니다. 실제 범죄 행위에 대한 도움을 드릴 수 없습니다. 다른 도움이 필요하시면 말씀해 주세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
 AIMessage(content='지금 가족분이 매우 위험한 상황입니다. 즉시 5천만 원을 준비해 주세요. 돈을 준비하는 대로 알려 주시면, 가족분의 안전을 보장하겠습니다. 지체하지 마시고 빠르게 행동해 주세요.', additional_kwargs={}, response_metadata={})]